In [15]:
import pandas as pd
from pymystem3 import Mystem
import spacy
nlp = spacy.load("ru_core_news_lg")

### Data Load

In [82]:
df_manual = []
with open("../edz_dataset.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.replace("\ufeff" ,"")
        line = line.replace("\n" ,"")
        df_manual.append([line[:-1], 1])
        
with open("../edz_neg_dataset.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.replace("\ufeff" ,"")
        line = line.replace("\n" ,"")
        df_manual.append([line[:-1], 0])
df_manual = pd.DataFrame(df_manual, columns=["text", "label"])


In [6]:
df_manual

,text,label
0,завтра буду учить английский весь ден,1
1,Завтра я пойду на работ,1
2,послезавтра мы планируем ехать на дач,1
3,к вечеру я планирую закончить отче,1
4,в воскресенье после обеда мы пойдем в рестора,1
...,...,...
216,Где ты будешь в 15:30 завтра,0
217,Король Генрих по утрам делает зарядку с королево,0
218,Хочешь утром посмотреть как пасут коров на наб...,0
219,Когда в следующем году проведут конференцию по...,0


### Tags review

#### SpaCy

In [8]:
tags = []
doc = nlp(df_manual.loc[22, "text"])
for token in doc:
    tags.append([token.text, token.lemma_, token.ent_type_, token.pos_, token.tag_, token.dep_, token.head])
postok = {str(i): list(doc).index(i) for i in doc}
pd.DataFrame(tags, columns = ["Token", "lemma", "entity", "full", "entity", "dep type", "head"])

,Token,lemma,entity,full,entity,dep type,head
0,к,к,,ADP,ADP,case,пяти
1,пяти,пять,,NUM,NUM,nummod,утра
2,утра,утро,,NOUN,NOUN,obl,завершится
3,хакатон,хакатон,,PROPN,PROPN,nsubj,завершится
4,завершится,завершиться,,VERB,VERB,ROOT,завершится
5,и,и,,CCONJ,CCONJ,cc,сдадим
6,мы,мы,,PRON,PRON,nsubj,сдадим
7,сдадим,сдать,,VERB,VERB,conj,завершится
8,результат,результат,,NOUN,NOUN,obj,сдадим


#### MyStem

In [16]:
stem = Mystem()
stem.analyze(df_manual.loc[22, "text"])

Installing mystem to /Users/R1303-M-4-Stud/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-macosx.tar.gz


[{'analysis': [{'lex': 'к', 'wt': 0.9999551798, 'gr': 'PR='}], 'text': 'к'},
 {'text': ' '},
 {'analysis': [{'lex': 'пять', 'wt': 1, 'gr': 'NUM=(пр|дат|род)'}],
  'text': 'пяти'},
 {'text': ' '},
 {'analysis': [{'lex': 'утро',
    'wt': 1,
    'gr': 'S,сред,неод=(вин,мн|род,ед|им,мн)'}],
  'text': 'утра'},
 {'text': ' '},
 {'analysis': [{'lex': 'хакатон',
    'wt': 0.2835818696,
    'qual': 'bastard',
    'gr': 'S,муж,неод=(вин,ед|им,ед)'}],
  'text': 'хакатон'},
 {'text': ' '},
 {'analysis': [{'lex': 'завершаться',
    'wt': 1,
    'gr': 'V,нп=непрош,ед,изъяв,3-л,сов'}],
  'text': 'завершится'},
 {'text': ' '},
 {'analysis': [{'lex': 'и', 'wt': 0.9999770357, 'gr': 'CONJ='}], 'text': 'и'},
 {'text': ' '},
 {'analysis': [{'lex': 'мы', 'wt': 1, 'gr': 'SPRO,мн,1-л=им'}], 'text': 'мы'},
 {'text': ' '},
 {'analysis': [{'lex': 'сдавать',
    'wt': 1,
    'gr': 'V,пе=(мн,пов,1-л,сов|непрош,мн,изъяв,1-л,сов)'}],
  'text': 'сдадим'},
 {'text': ' '},
 {'analysis': [{'lex': 'результат',
    'wt':

#### Natasha

In [17]:
import natasha

In [20]:
%script false
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)
text = df.loc[23, "text"]
doc = Doc(text)

UsageError: Line magic function `%script` not found (But cell magic `%%script` exists, did you mean that instead?).


### BoW

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [83]:
model_bow = CountVectorizer()
X = model_bow.fit_transform(df_manual.text).toarray()
y = df_manual.label
x_train, x_test, y_train, y_test = train_test_split(X, y, shuffle=True)

In [104]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

In [105]:
from sklearn.metrics import classification_report
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.98      0.86        55
           1       0.97      0.70      0.81        56

    accuracy                           0.84       111
   macro avg       0.87      0.84      0.83       111
weighted avg       0.87      0.84      0.83       111



In [101]:
model.predict(model_bow.transform(["Я покушал колбаску с сыром"]))

array([0])

In [102]:
model.predict(model_bow.transform(["В эту пятницу у меня занятия танцами "]))

array([1])

In [76]:
df


,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,Общество
1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир
3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика
4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество
...,...,...,...,...,...
995,https://lenta.ru/news/2018/12/08/skater/,Американский фигурист «воткнул» партнершу в ле...,Американский фигурист Тимоти Ледук уронил парт...,Спорт,Зимние виды
996,https://lenta.ru/news/2018/12/08/vred/,В новой нефтяной сделке нашли вред для России,Новое соглашение членов Организации стран-эксп...,Экономика,Госэкономика
997,https://lenta.ru/news/2018/12/08/9m729/,Предсказано будущее ракеты 9М729 после ультима...,Ответом России на ультиматум США по Договору о...,Наука и техника,Оружие
998,https://lenta.ru/news/2018/12/08/paris/,В охваченный протестами Париж вошла бронетехника,"В Париж, где продолжаются антиправительственны...",Мир,Общество


In [81]:
df = pd.read_csv("/Users/R1303-M-4-Stud/PlanningLooker/lenta-ru-news_csv/lenta-ru-news.csv", nrows=200)
add_rows = []
for i in df["title"]:
    add_rows.append(i)
with open("/Users/R1303-M-4-Stud/PlanningLooker/edz_neg_dataset.txt", "a") as f:
    for i in add_rows:
        f.write(i+"\n")